In [7]:
# AutoDL官方学术资源加速
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [8]:
import sys
import os

# 添加项目根目录到Python路径
project_root = "/home/cuipeng/Gemma"
sys.path.append(project_root)

# 现在可以正常导入src下的模块
from src.core.model.model_initializer import initialize_model_and_tokenizer
from src.core.utils.model_utils import generate_response, apply_chat_template

In [9]:
from datasets import load_dataset # type: ignore
import pandas as pd # type: ignore
import numpy as np # type: ignore
from typing import List, Dict # type: ignore
import json # type: ignore
import os # type: ignore
from tqdm import tqdm # type: ignore
import random
from openai import OpenAI # type: ignore

# config.py
from dotenv import load_dotenv # type: ignore

# 加载 .env 文件
load_dotenv()

# 读取
ZetaTechs_api_key = os.getenv('ZETATECHS_API_KEY')
ZetaTechs_api_base = os.getenv('ZETATECHS_API_BASE')

# 初始化OpenAI客户端
client = OpenAI(api_key=ZetaTechs_api_key, base_url=ZetaTechs_api_base)

In [10]:
class BaseDatasetBuilder:
    def __init__(self):
        """初始化基础数据集构建器"""
        self.datasets = {}
        self.processed_data = []

    def load_base_datasets(self):
        """加载基础阶段所需的数据集"""
        print("正在加载基础数据集...")
        
        try:
            # 通用指令数据集
            self.datasets['belle'] = load_dataset("BelleGroup/train_1M_CN", trust_remote_code=True)
            print("✓ Belle数据集加载完成")
            
            # Alpaca中文数据集
            self.datasets['alpaca'] = load_dataset('silk-road/alpaca-data-gpt4-chinese')
            print("✓ Alpaca中文数据集加载完成")
            
        except Exception as e:
            print(f"加载数据集时出错: {e}")
            raise
    
    def preprocess_data(self, max_samples_per_dataset=50000):
        """预处理各个数据集"""
        for dataset_name, dataset in self.datasets.items():
            print(f"处理 {dataset_name} 数据集...")
            
            if dataset_name == 'belle':
                processed = self._process_belle(dataset, max_samples_per_dataset)
            elif dataset_name == 'alpaca':
                processed = self._process_alpaca(dataset, max_samples_per_dataset)
                
            self.processed_data.extend(processed)
            print(f"✓ {dataset_name} 处理完成，添加 {len(processed)} 条数据")
    
    def _process_belle(self, dataset, max_samples):
        """处理Belle数据集"""
        samples = dataset["train"][:max_samples]
        processed_data = []

        for i in range(len(samples['instruction'])):
            messages = [
                {"role": "user", "content": samples['instruction'][i]},
                {"role": "model", "content": samples['output'][i]}
            ]
            
            processed_data.append({'messages': messages})
            
            # 添加进度显示
            # if (i + 1) % 1000 == 0:
            #     print(f"Belle数据集处理进度: {i + 1}/{len(samples)}")
        
        return processed_data
    
    def _process_alpaca(self, dataset, max_samples):
        """处理Alpaca数据集"""
        samples = dataset["train"][:max_samples]
        processed_data = []

        for i in range(len(samples['instruction_zh'])):
            # 合并instruction和input（如果有）
            user_content = (f"{samples['instruction_zh'][i]}\n\n"
                          f"输入：{samples['input_zh'][i]}" if samples['input_zh'][i] 
                          else samples['instruction_zh'][i])
            
            messages = [
                {"role": "user", "content": user_content},
                {"role": "model", "content": samples['output_zh'][i]}
            ]
            
            processed_data.append({'messages': messages})
            
            # 添加进度显示
            # if (i + 1) % 1000 == 0:
            #     print(f"Alpaca数据集处理进度: {i + 1}/{len(samples)}")
        
        return processed_data
    
    def format_for_training(self):
        """将数据格式化为训练所需的格式"""
        formatted_data = []
        
        for item in self.processed_data:
            dialogue_str = apply_chat_template(item['messages'])
            formatted_data.append({
                'text': dialogue_str,
                'type': 'instruction'
            })
            
        return formatted_data
    
    def save_dataset(self, formatted_data, output_path='./stage1/data_raw', train_ratio=0.8, valid_size=1000):
        """保存处理后的数据集
        
        Args:
            formatted_data: 格式化后的数据
            output_path: 输出路径
            train_ratio: 训练集占比
            valid_size: 验证集最终保留的数据量
        """
        os.makedirs(output_path, exist_ok=True)
        
        # 随机打乱数据
        np.random.shuffle(formatted_data)
        
        # 按原比例划分训练集和验证集
        split_idx = int(len(formatted_data) * train_ratio)
        train_data = formatted_data[:split_idx]
        valid_data = formatted_data[split_idx:]
        
        # 从验证集中随机选择1000条数据
        if len(valid_data) > valid_size:
            # 随机选择指定数量的验证数据
            valid_indices = np.random.choice(len(valid_data), valid_size, replace=False)
            valid_data = [valid_data[i] for i in valid_indices]
        
        # 保存训练集
        train_path = os.path.join(output_path, 'train.json')
        with open(train_path, 'w', encoding='utf-8') as f:
            json.dump(train_data, f, ensure_ascii=False, indent=2)
            
        # 保存验证集
        valid_path = os.path.join(output_path, 'valid.json')
        with open(valid_path, 'w', encoding='utf-8') as f:
            json.dump(valid_data, f, ensure_ascii=False, indent=2)
            
        print(f"\n数据集已保存：")
        print(f"- 训练集: {train_path} ({len(train_data)} 条数据)")
        print(f"- 验证集: {valid_path} ({len(valid_data)} 条数据)")
        print(f"- 原始验证集大小: {len(formatted_data) - split_idx}")
        print(f"- 最终验证集大小: {len(valid_data)}")
        
        # 输出数据集统计信息
        self._print_dataset_stats(formatted_data)
    
    def _print_dataset_stats(self, data):
        """打印数据集统计信息"""
        print("\n数据集统计信息:")
        print(f"总数据量: {len(data)}")

In [11]:
def main():
    # 1. 初始化数据集构建器
    builder = BaseDatasetBuilder()
    
    try:
        # 2. 加载数据集
        builder.load_base_datasets()
        
        # 3. 处理数据集
        builder.preprocess_data(max_samples_per_dataset=50000)
        
        # 4. 格式化数据
        formatted_data = builder.format_for_training()     
        
        # 5. 保存数据集
        builder.save_dataset(formatted_data)
        
        print("\n基础阶段数据集构建完成！")
        
    except Exception as e:
        print(f"构建数据集时出错: {e}")

In [12]:
if __name__ == "__main__":
    main()

正在加载基础数据集...
✓ Belle数据集加载完成
✓ Alpaca中文数据集加载完成
处理 belle 数据集...
✓ belle 处理完成，添加 50000 条数据
处理 alpaca 数据集...
✓ alpaca 处理完成，添加 50000 条数据

数据集已保存：
- 训练集: ./stage1/data_raw/train.json (80000 条数据)
- 验证集: ./stage1/data_raw/valid.json (1000 条数据)
- 原始验证集大小: 20000
- 最终验证集大小: 1000

数据集统计信息:
总数据量: 100000

基础阶段数据集构建完成！
